In [2]:
#default_exp modeling.trainingfastai

In [3]:
#hide
from nbdev.showdoc import *
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Training Functionality

These functions are designed to help with anything in the training stage of the ML life cycle.

In [4]:
#export
import os
import logging
import gc

from sdsde.wrapper.azurewrapper import blob_pusher
from fastai.basics import *
from fastai.tabular.all import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## DL Basic Model Training

### ``train_fastai_tabular_model``

In [5]:
#export


def train_fastai_tabular_model(dl, layer_sizes: list, metrics: list,
                               epochs: int, wd: float, save_metric_monitor: str, save_comp,
                               model_name: str, early_metric_monitor, early_min_delta: float,
                               early_comp, patience: int, project_location: str,
                               tab_config: dict = None, y_range: float = None,
                               plot: bool = True):
    """
    Train a neural network and return leaner

    Args:
    * dl (TabularDataLoader): DataLoader needed
    * layer_sizes (list): Model Layer Sizes
    * metrics (list): Metrics of interest to watch while training
    * epochs (int): Number of epochs
    * wd (float): Weight decay
    * save_metric_monitor (str): Model metric to watch and save only when metric improves
    * save_comp ([type]): Metric should be np.greater or np.lesser depends on metric
    * model_name (str): Model Name
    * early_metric_monitor ([type]): Early stopping metric to stop training the model when improvement stops
    * early_min_delta (float): How much the metric has to improve by unless counter will begin for early stopping
    * early_comp ([type]): Metrics should be np.greater or np.lesser depends on metric
    * patience (int): How many epochs to wait till improve decays
    * project_location (str): Where the model will be stored for all purposes of training
    * tab_config (dict): Convenient function to add drop out/ or remove some standard best practices
        defaults: ps=None, embed_p=0.0, y_range=None, use_bn=True, bn_final=False, bn_cont=True, act_cls=ReLU(inplace=True)
    * y_range (float, optional): Regression only giving a range the y variable can be. Defaults to None.
    * plot (bool, optional): plot information to be able to see graphs. Defaults to True.

    Returns:
    * list: Learner, probabilities, acutals, loss
    """
    learn = tabular_learner(dl, layers=layer_sizes, path=project_location,
                            y_range=y_range, metrics=metrics, config=tab_config)
    cbs = [SaveModelCallback(monitor=save_metric_monitor, comp=save_comp, fname=model_name),
           EarlyStoppingCallback(monitor=early_metric_monitor, comp=np.greater,
                                 min_delta=early_min_delta, patience=patience)]
    if plot is True:
        cbs.append(ShowGraphCallback())
    lr_suggestion = learn.lr_find()
    logger.info(f'Learning Rate Suggestion {lr_suggestion}')
    gc.collect()
    os.makedirs(project_location, exist_ok=True)
    learn.fit_one_cycle(epochs, lr_suggestion[0], wd=wd, cbs=cbs)
    gc.collect()
    probs, y, loss = learn.get_preds(dl=dl.valid, with_loss=True)
    return learn, probs, y, loss

In [6]:
show_doc(train_fastai_tabular_model)

<h4 id="train_fastai_tabular_model" class="doc_header"><code>train_fastai_tabular_model</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>train_fastai_tabular_model</code>(**`dl`**, **`layer_sizes`**:`list`, **`metrics`**:`list`, **`epochs`**:`int`, **`wd`**:`float`, **`save_metric_monitor`**:`str`, **`save_comp`**, **`model_name`**:`str`, **`early_metric_monitor`**, **`early_min_delta`**:`float`, **`early_comp`**, **`patience`**:`int`, **`project_location`**:`str`, **`tab_config`**:`dict`=*`None`*, **`y_range`**:`float`=*`None`*, **`plot`**:`bool`=*`True`*)

Train a neural network and return leaner

Args:
* dl (TabularDataLoader): DataLoader needed
* layer_sizes (list): Model Layer Sizes
* metrics (list): Metrics of interest to watch while training
* epochs (int): Number of epochs
* wd (float): Weight decay
* save_metric_monitor (str): Model metric to watch and save only when metric improves
* save_comp ([type]): Metric should be np.greater or np.lesser depends on metric
* model_name (str): Model Name
* early_metric_monitor ([type]): Early stopping metric to stop training the model when improvement stops
* early_min_delta (float): How much the metric has to improve by unless counter will begin for early stopping
* early_comp ([type]): Metrics should be np.greater or np.lesser depends on metric
* patience (int): How many epochs to wait till improve decays
* project_location (str): Where the model will be stored for all purposes of training
* tab_config (dict): Convenient function to add drop out/ or remove some standard best practices
    defaults: ps=None, embed_p=0.0, y_range=None, use_bn=True, bn_final=False, bn_cont=True, act_cls=ReLU(inplace=True)
* y_range (float, optional): Regression only giving a range the y variable can be. Defaults to None.
* plot (bool, optional): plot information to be able to see graphs. Defaults to True.

Returns:
* list: Learner, probabilities, acutals, loss

## Model Movement

### ``save_fastai_model_to_data_lake``

In [13]:
#export


def save_fastai_model_to_data_lake(learner, file_name: str, path: str, container: str,
                                   connection_str: str, local_path: str = '', overwrite=False):
    """
    Save a fastai model to data lake

    Args:
    * learner ([type]): fastai learner
    * file_name (str): file name
    * path (str): blob path location
    * container (str): container name
    * connection_str (str): azure connectioon string
    * local_path (str, optional): local file path. Defaults to ''.
    * overwrite (bool, optional): overwrite file. Defaults to False.
    """
    logger.info(f'saving learner at {os.path.join(learner.path, file_name)}')
    os.makedirs(learner.path, exist_ok=True)
    learner.export(fname=file_name)
    logger.info(f'Pushing Learner Object to Azure: {os.path.join(path, file_name)}')
    blob_pusher(container_name=container,
                connection_str=connection_str,
                file_path=[os.path.join(learner.path, file_name)],
                blob_dest=[path],
                overwrite=overwrite)

In [14]:
show_doc(save_fastai_model_to_data_lake)

<h4 id="save_fastai_model_to_data_lake" class="doc_header"><code>save_fastai_model_to_data_lake</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>save_fastai_model_to_data_lake</code>(**`learner`**, **`file_name`**:`str`, **`path`**:`str`, **`container`**:`str`, **`connection_str`**:`str`, **`local_path`**:`str`=*`''`*, **`overwrite`**=*`False`*)

Save a fastai model to data lake

Args:
* learner ([type]): fastai learner
* file_name (str): file name
* path (str): blob path location
* container (str): container name
* connection_str (str): azure connectioon string
* local_path (str, optional): local file path. Defaults to ''.
* overwrite (bool, optional): overwrite file. Defaults to False.

# Create

In [15]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_azure.ipynb.
Converted 02_utils_dataframes.ipynb.
Converted 02_utils_parseyaml.ipynb.
Converted 02_utils_stfp.ipynb.
Converted 02_utils_traininghelpers.ipynb.
Converted 02_utils_traininghelpers_fastai.ipynb.
Converted 03_dstools_preparedata.ipynb.
Converted 04_snowflake_copyinto.ipynb.
Converted 04_snowflake_copyinto2.ipynb.
Converted 04_snowflake_query.ipynb.
Converted 05_azure_wrappers.ipynb.
Converted 06_modeling_inference.ipynb.
Converted 06_modeling_inference_fastai.ipynb.
Converted 06_modeling_premodel.ipynb.
Converted 06_modeling_preprocessing.ipynb.
Converted 06_modeling_preprocessing_fastai.ipynb.
Converted 06_modeling_training.ipynb.
Converted 06_modeling_training_fastai.ipynb.
Converted 07_Binary_Classification_Fastai_Example_Notebook.ipynb.
Converted 08_yaml_ingestion_binary_classification.ipynb.
Converted index.ipynb.
